https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564

http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/

https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564

https://kwotsin.github.io/tech/2017/01/29/tfrecords.html

https://medium.com/ymedialabs-innovation/how-to-use-tfrecord-with-datasets-and-iterators-in-tensorflow-with-code-samples-ffee57d298af

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
arr = np.array([[9.0, 9.7], [1,2]])
arr.shape

In [ ]:
data = {
    'Age': 29,
    'Movie': ['The Shawshank Redemption', 'Fight Club'],
    'Movie Ratings': [9.0, 9.7],
    'Suggestion': 'Inception',
    'Suggestion Purchased': 1.0,
    'Purchase Price': 9.99
}


In [ ]:
# # Example -> Final Wrapper. cons
#     |_tf.train.Features
#         |_tf.train.Feature
#             |_tf.train.Int64List
#         |_tf.train.Feature
#             |_tf.train.BytesList
#         |_tf.train.Feature
#             |_tf.train.FloatList

In [ ]:
# Create the Example
feature_1 = {
    'Age': tf.train.Feature(
        int64_list=tf.train.Int64List(value=[data['Age']])),
    'Movie': tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[m.encode('utf-8') for m in data['Movie']])),
    'Movie Ratings': tf.train.Feature(
        float_list=tf.train.FloatList(value=data['Movie Ratings'])),
    'Suggestion': tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[data['Suggestion'].encode('utf-8')])),
    'Suggestion Purchased': tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[data['Suggestion Purchased']])),
    'Purchase Price': tf.train.Feature(
        float_list=tf.train.FloatList(value=[data['Purchase Price']]))
    }
feature_2 = {
    'Age': tf.train.Feature(
        int64_list=tf.train.Int64List(value=[data['Age']])),
    'Movie': tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[m.encode('utf-8') for m in data['Movie']])),
    'Movie Ratings': tf.train.Feature(
        float_list=tf.train.FloatList(value=data['Movie Ratings'])),
    'Suggestion': tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[data['Suggestion'].encode('utf-8')])),
    'Suggestion Purchased': tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[data['Suggestion Purchased']])),
    'Purchase Price': tf.train.Feature(
        float_list=tf.train.FloatList(value=[data['Purchase Price']]))
    }

example1 = tf.train.Example(features=tf.train.Features(feature=feature_1))
example2 = tf.train.Example(features=tf.train.Features(feature=feature_2))

examples = [example1, example2]

print(example1)

In [ ]:
# Write TFrecord file
with tf.python_io.TFRecordWriter('customer_1.tfrecord') as writer:
    for example in examples:
        writer.write(example.SerializeToString())

In [ ]:
# Read and print data:
sess = tf.InteractiveSession()

# Read TFRecord file
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['customer_1.tfrecord'])

_, serialized_example = reader.read(filename_queue)

# Define features
read_features = {
    'Age': tf.FixedLenFeature([], dtype=tf.int64),
    'Movie': tf.VarLenFeature(dtype=tf.string),
    'Movie Ratings': tf.VarLenFeature(dtype=tf.float32),
    'Suggestion': tf.FixedLenFeature([], dtype=tf.string),
    'Suggestion Purchased': tf.FixedLenFeature([], dtype=tf.float32),
    'Purchase Price': tf.FixedLenFeature([], dtype=tf.float32)}

# Extract features from serialized data
read_data = tf.parse_single_example(serialized=serialized_example,
                                    features=read_features)

# Many tf.train functions use tf.train.QueueRunner,
# so we need to start it before we read
tf.train.start_queue_runners(sess)

# Print features
for name, tensor in read_data.items():
    print('{}: {}'.format(name, tensor.eval()))

### Test shabda Features

In [ ]:
# Read and print data:
sess = tf.InteractiveSession()

# Read TFRecord file
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['/home/mageswarand/vitaFlow/TEDLiumTestDataset/preprocessed_data/test/tfrecords/42.tfrecord'])

_, serialized_example = reader.read(filename_queue)

# Define features
read_features = {
                'speech_mix': tf.VarLenFeature(tf.float32),
                'VAD': tf.FixedLenFeature([12900], tf.int64),
                'Y': tf.FixedLenFeature([2*12900], tf.int64),
                'FPS': tf.FixedLenFeature([], tf.int64),
                'NEFF': tf.FixedLenFeature([], tf.int64),
            }

# Extract features from serialized data
read_data = tf.parse_single_example(serialized=serialized_example,
                                    features=read_features)

# Many tf.train functions use tf.train.QueueRunner,
# so we need to start it before we read
tf.train.start_queue_runners(sess)

# Print features
for name, tensor in read_data.items():
    print('{}: {}'.format(name, tensor.eval()))

In [ ]:
def decode( serialized_example):
    # 1. define a parser
    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'speech_mix': tf.FixedLenFeature([100 * 129], tf.float32),
            'VAD': tf.FixedLenFeature([100 * 129], tf.int64),
            'Y': tf.FixedLenFeature([2*100 * 129], tf.int64),
            'FPS': tf.FixedLenFeature([], tf.int64),
            'NEFF': tf.FixedLenFeature([], tf.int64),
        })
    FPS = tf.cast(features['FPS'], tf.int32)
    NEFF = tf.cast(features['NEFF'], tf.int32)
    sample_mix = tf.reshape(tf.cast(features['speech_mix'], tf.float32), shape=[100, 129])
    VAD = tf.reshape(tf.cast(features['VAD'], tf.float32), shape=[100, 129])
    Y = tf.reshape(tf.cast(features['Y'], tf.float32), shape=[100, 129, 2])

    return {"sample": sample_mix, "vad": VAD}, Y

In [ ]:
import glob
files = glob.glob(os.path.join("/home/mageswarand/vitaFlow/TEDLiumTestDataset/preprocessed_data/test", "tfrecords/*.tfrecord"))

In [ ]:
os.path.join("/home/mageswarand/vitaFlow/TEDLiumTestDataset/preprocessed_data/test", "tfrecords/*.tfrecord")

In [ ]:
files

In [ ]:
# Initialize all tfrecord paths
dataset = tf.data.TFRecordDataset(['/home/mageswarand/vitaFlow/TEDLiumTestDataset/preprocessed_data/test/tfrecords/42.tfrecord'])
dataset = dataset.map(decode)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            data_record = sess.run(next_element)
            print(data_record)
    except:
        pass